# install latest hg-python etc as per Trevor's example:

In [1]:
# # install (latest) version of manzt/hg from GitHub
# # commented, as you do not want to reinstall it every time
# !pip install git+https://github.com/manzt/hg.git@853c4b7ca54e8e2e1cea6b42878653c159234ade
# !pip install clodius 

# also install latest cooltools from master-branch ...

In [2]:
%load_ext autoreload
%autoreload 2
import cooltools
from cooltools.sandbox import obs_over_exp_cooler
import cooler
import numpy as np
import bioframe
import pandas as pd
# from typing import TypeVar
# PandasDataFrame = TypeVar('pandas.core.frame.DataFrame')

# import sample cooler
it's a small cooler with just 2 chromosomes chr2 and chr17 using a limited resolution "ladder" 1,10,100,1000 kb ...

In [3]:
cool_file = cooltools.download_data("HFF_MicroC", cache=True, data_dir='./')
cool_file

'./test.mcool'

# precalculate expected for every resolution of the mcool ...
an in-memory held dictionary of "expected" DataFrames for every resolution that is in the provided mcool

In [4]:
# Use bioframe to fetch the genomic features from the UCSC.
hg38_chromsizes = bioframe.fetch_chromsizes('hg38')
hg38_cens = bioframe.fetch_centromeres('hg38')
# create a view with chromosome arms using chromosome sizes and definition of centromeres
hg38_arms = bioframe.make_chromarms(hg38_chromsizes,  hg38_cens)

In [5]:
# dictionary to hold an expected DataFrame for every resolution in the input mcool:
exp_dict = {}
for res in [1000000, 100000, 10000, 1000]:
    print(f"working on resolution {res} ...")
    clr = cooler.Cooler(f"{cool_file}::/resolutions/{res}")

    # generate bins table with weights=1, and NaN for bad bins ...
    bins_oe = clr.bins()[:]
    _bad_mask = bins_oe["weight"].isna()
    bins_oe["weight"] = 1.
    bins_oe.loc[_bad_mask,"weight"] = np.nan
    
    # select only those chromosomes available in cooler
    hg38_arms = hg38_arms[hg38_arms.chrom.isin(clr.chromnames)].reset_index(drop=True)

    # re-calculate full expected (cis + trans) at higher resolution
    exp_dict[res] = obs_over_exp_cooler.expected_full(
        clr,
        view_df=hg38_arms,
        smooth_cis=False,
        aggregate_trans=False,
        expected_column_name="expected",
        ignore_diags=0,
        nproc=8,
    )

working on resolution 1000000 ...


INFO:root:Done calculating cis expected in 0.446 sec ...
INFO:root:Done calculating trans expected in 0.392 sec ...
INFO:root:Returning combined expected DataFrame.


working on resolution 100000 ...


INFO:root:Done calculating cis expected in 1.565 sec ...
INFO:root:Done calculating trans expected in 1.446 sec ...
INFO:root:Returning combined expected DataFrame.


working on resolution 10000 ...


INFO:root:Done calculating cis expected in 5.584 sec ...
INFO:root:Done calculating trans expected in 3.960 sec ...
INFO:root:Returning combined expected DataFrame.


working on resolution 1000 ...


INFO:root:Done calculating cis expected in 14.986 sec ...
INFO:root:Done calculating trans expected in 8.852 sec ...
INFO:root:Returning combined expected DataFrame.


# create custom `hg.tilesets.LocalTileset` from `clodius.tiles`

original Trevor's notebook https://gist.github.com/manzt/62be0e7c8a2c47f8c517d567dc9f362c

The `higlass-server` relies on `clodius` for keeping a consistent API for accessing datasets as `Tilesets`. Every clodius tiles implementation includes a `tiles` and `tilset_info` implementation. The `hg.tilesets.LocalTileset` is a _very_ thin layer used to wrap these implementations for `hg`. 

In general, it is used to bind the filepath to the `clodius` functions and set a `uid` for uniquely identifying the tileset on the server. The function `create_cooler_tileset` is adapated from `hg`'s own cooler implementation to demonstrate how to hook into the server.

In [6]:
import hg
# hg.server.enable_proxy()
# import clodius.tiles.cooler
import functools
import uuid

# this is a slightly modified https://github.com/higlass/clodius/blob/1ed7211643bdee1a6fa3c17c14db8158c1fce156/clodius/tiles/cooler.py
# i.e. a re-implementation of the current cooler-tileset fetcher from clodius
# only get_data function was modified +expected,view arguments were added into tileset-related functions
import oe

def tiles(filepath: str, expected: dict, view_df: pd.DataFrame):
    # TODO: re-write with custom tiles handler
    # Look at: https://github.com/higlass/clodius/blob/develop/clodius/tiles/cooler.py for implementation details
    return functools.partial(oe.tiles, filepath, expected, view_df)

def tileset_info(filepath: str):
    # probably don't need to override this function from base cooler (tileset info likely the same)
    return functools.partial(oe.tileset_info, filepath)

def create_cooler_tileset(filepath: str, expected: dict, view_df: pd.DataFrame) -> hg.tilesets.LocalTileset:
    """Creates a Tileset to add to the hg.server"""
    return hg.tilesets.LocalTileset(
        datatype="matrix",
        tiles=tiles(filepath, expected, view_df),
        info=tileset_info(filepath),
        uid=str(uuid.uuid4()),  # uniquely identifies tileset for server
        name="xxx",
    )

# tileset = create_cooler_tileset("/home/sergpolly/Desktop/4dn2/fun.mcool")
tileset = create_cooler_tileset(
    "test.mcool",
    exp_dict,
    hg38_arms[hg38_arms.chrom.isin(clr.chromnames)].reset_index(drop=True),
)
# tileset = create_cooler_tileset("test-oe.mcool")
tileset;

# add `tileset` instance to `hg.server`

In [7]:
tileset_resource = hg.server.add(tileset, port=10000)
tileset_resource; # can be used now in hg

In [8]:
tileset_resource.tileset.info()

{'resolutions': (1000, 10000, 100000, 1000000),
 'transforms': [{'name': 'ICE', 'value': 'weight'}],
 'max_pos': [325450970, 325450970],
 'min_pos': [1, 1],
 'chromsizes': [['chr2', 242193529], ['chr17', 83257441]]}

# use in `hg`

In [9]:
# our main "track" - the heatmap with some custom options - fixed value Scale and blue to red colorscale...
track = tileset_resource \
    .track("heatmap") \
    .opts(valueScaleMin=0.1,valueScaleMax=10,colorRange=["blue","white","red"])

v1 = hg.view(
    hg.track("top-axis"),
    track,
    width=6
)
# best to switch to bwr colormap and limit vmin,vmax to a symmetric colormap around "1"
v1

View(layout=Layout(x=0, y=0, w=6, h=6, moved=None, static=None), tracks=Tracks[TrackT](left=None, right=None, top=[EnumTrack(tilesetUid=None, server=None, type='top-axis', uid='TMBPCxHeT1aBYM2GOBT9xg', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], bottom=None, center=[HeatmapTrack(tilesetUid='90c47799-1b36-4a06-8cfd-08cb07731c91', server='http://localhost:10000/api/v1/', type='heatmap', uid='eLCi87jhQ7S-mMjR6mi4KA', width=None, height=None, options={'name': 'xxx', 'valueScaleMin': 0.1, 'valueScaleMax': 10, 'colorRange': ['blue', 'white', 'red']}, data=None, position=None, transforms=None)], whole=None, gallery=None), uid='f5U05y9iTfqS177ij1619Q', autocompleteSource=None, chromInfoPath=None, genomePositionSearchBox=None, genomePositionSearchBoxVisible=None, initialXDomain=None, initialYDomain=None, overlays=None, selectionView=None, zoomFixed=None, zoomLimits=(1, None))